In [1]:
import pandas as pd
import numpy as np
import re
import time
import ast
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline

from sklearn.multiclass import OneVsRestClassifier
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

from gensim import corpora, models, similarities

# Linear Models 
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import linear_model
from sklearn.naive_bayes import MultinomialNB

from tqdm import tqdm

import keras
from keras import optimizers
from keras import backend as K
from keras import regularizers
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout, Flatten, LSTM, Input, RNN
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.utils import plot_model
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.callbacks import EarlyStopping



C:\Anaconda2\envs\py36\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
nameTrainCSV = 'trainWithListOfCleanWords'
nameTestCSV = 'testWithListOfCleanWords'

train = pd.read_csv('../data/processed/' + nameTrainCSV + '.csv', encoding='utf-8')
train['BagOfWords'] = dict
train.head(10)


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"['explanation', 'edits', 'made', 'username', '...",explanation edits made username hardcore metal...,<class 'dict'>
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"['aww', 'match', 'background', 'colour', 'seem...",aww match background colour seemingly stuck th...,<class 'dict'>
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"['hey', 'man', 'really', 'trying', 'edit', 'wa...",hey man really trying edit war guy constantly ...,<class 'dict'>
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"['make', 'real', 'suggestion', 'improvement', ...",make real suggestion improvement wondered sect...,<class 'dict'>
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"['sir', 'hero', 'chance', 'remember', 'page']",sir hero chance remember page,<class 'dict'>
5,5,00025465d4725e87,"""\n\nCongratulations from me as well, use the ...",0,0,0,0,0,0,"['congratulation', 'well', 'use', 'tool', 'wel...",congratulation well use tool well talk,<class 'dict'>
6,6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0,"['cocksucker', 'piss', 'around', 'work']",cocksucker piss around work,<class 'dict'>
7,7,00031b1e95af7921,Your vandalism to the Matt Shirvington article...,0,0,0,0,0,0,"['vandalism', 'matt', 'shirvington', 'article'...",vandalism matt shirvington article reverted pl...,<class 'dict'>
8,8,00037261f536c51d,Sorry if the word 'nonsense' was offensive to ...,0,0,0,0,0,0,"['sorry', 'word', 'nonsense', 'offensive', 'an...",sorry word nonsense offensive anyway intending...,<class 'dict'>
9,9,00040093b2687caa,alignment on this subject and which are contra...,0,0,0,0,0,0,"['alignment', 'subject', 'contrary', 'dulithgow']",alignment subject contrary dulithgow,<class 'dict'>


In [3]:
start = time.time()
for x in range(len(train)):
    train.set_value(col='listOfCleanWords',
                index=x,
                value=ast.literal_eval(train["listOfCleanWords"][x]))
    train.set_value(col='BagOfWords',
                index=x,
                value=Counter(train["listOfCleanWords"][x]))
end = time.time()
print(end - start)

14.836838722229004


In [4]:
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,listOfCleanWords,cleanWordsAsText,BagOfWords
0,0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanation, edits, made, username, hardcore,...",explanation edits made username hardcore metal...,"{'explanation': 1, 'edits': 1, 'made': 1, 'use..."
1,1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[aww, match, background, colour, seemingly, st...",aww match background colour seemingly stuck th...,"{'aww': 1, 'match': 1, 'background': 1, 'colou..."
2,2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, really, trying, edit, war, guy, con...",hey man really trying edit war guy constantly ...,"{'hey': 1, 'man': 1, 'really': 1, 'trying': 1,..."
3,3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[make, real, suggestion, improvement, wondered...",make real suggestion improvement wondered sect...,"{'make': 1, 'real': 1, 'suggestion': 1, 'impro..."
4,4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page]",sir hero chance remember page,"{'sir': 1, 'hero': 1, 'chance': 1, 'remember':..."


In [5]:
# MULTICLASS PREDICTION

In [6]:
classLabel = {
    0: "neutral",
    1: "toxic",
    2 : "severe_toxic",
    3 : "obscene",
    4 : "threat",
    5 : "insult",
    6 : "identity_hate" 
}


In [7]:
y = np.empty((len(train['cleanWordsAsText']),),dtype=object)
allTextToxicTrain = dict()
for idx in classLabel:
    if classLabel[idx] != "neutral":
        T = np.where(train[classLabel[idx]] == 1)[0]
        allTextToxicTrain[idx] = T
        for i in T:
            if y[i] is None:
                y[i] = [idx]                
            else:
                y[i].append(idx)
indxsOfNeutralTexts = np.where(y == None) 
y[indxsOfNeutralTexts] = [[0]]
indxsOfNeutralTexts = indxsOfNeutralTexts[0]

allTextsNoToxicTrain = [str(train['cleanWordsAsText'][x]) for x in indxsOfNeutralTexts]

idxList = []
for i in allTextToxicTrain.keys():
    #allTextToxicTrain[i] = [str(train['cleanWordsAsText'][j]) for j in allTextToxicTrain[i]]
    idxList = np.unique(np.append(idxList, allTextToxicTrain[i]))
allTextToxicTrain = [str(train['cleanWordsAsText'][j]) for j in idxList]

In [8]:
# Test clasification
test = pd.read_csv('../data/processed/' + nameTestCSV + '.csv', encoding='utf-8')
test.head()


,Unnamed: 0,id,comment_text,listOfCleanWords,cleanWordsAsText
0,0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...,"['yo', 'bitch', 'ja', 'rule', 'succesful', 'ev...",yo bitch ja rule succesful ever whats hating s...
1,1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...,"['rfc', 'title', 'fine', 'imo']",rfc title fine imo
2,2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap...","['source', 'zawe', 'ashton', 'lapland']",source zawe ashton lapland
3,3,00017563c3f7919a,":If you have a look back at the source, the in...","['look', 'back', 'source', 'information', 'upd...",look back source information updated correct f...
4,4,00017695ad8997eb,I don't anonymously edit articles at all.,"['anonymously', 'edit', 'article']",anonymously edit article


In [9]:
allTrainText = [txt if txt is not np.nan else '' for txt in train['cleanWordsAsText']]
allTestText = [txt if txt is not np.nan else '' for txt in test['cleanWordsAsText']]
X_train = allTrainText
X_test = allTestText
yBinary = MultiLabelBinarizer().fit_transform(y)
y_train = yBinary


In [10]:
columns = ["idExp","numFeatures", "algorithm", "Nfolds", "accuaracy", "logloss", "fmeasure"]
dfTestResults = pd.DataFrame(columns=columns)

In [11]:
maxFeatures = 100000

In [12]:
# Vectorizer all text
tfidV = TfidfVectorizer(ngram_range=(1,6), max_features=maxFeatures)
X_train_tfid = tfidV.fit_transform(allTrainText)

# Fit all clasificators with tfid matrix
numFeatures = len(tfidV.get_feature_names())
print("NFeatures = " + str(numFeatures))
tfidVTest = TfidfVectorizer(vocabulary=tfidV.get_feature_names())
X_test_tfid = tfidVTest.fit_transform(X_test)

NFeatures = 100000


In [13]:
# BOW
bowFeatures = CountVectorizer(vocabulary=tfidV.get_feature_names())
X_train_bow = bowFeatures.fit_transform(allTrainText)

X_test_bow = bowFeatures.fit_transform(allTestText)

In [14]:
# Tokenize

tokenizer = Tokenizer(num_words=maxFeatures)
tokenizer.fit_on_texts(list(allTrainText))
X_train_tokenized_seq = tokenizer.texts_to_sequences(allTrainText)
X_test_tokenized_seq = tokenizer.texts_to_sequences(allTestText)


In [15]:
len(tokenizer.word_index)

158652

In [57]:
X_train_seq = pad_sequences(X_train_tokenized_seq)
X_test_seq = pad_sequences(X_test_tokenized_seq, maxlen=len(X_train_seq[0]))

1250

In [56]:
X_train_seq[0]

array([   0,    0,    0, ...,    2,   69, 2933])

In [17]:
# MODEL CNN
numClases = 7

#training params
batch_size = 512 
num_epochs = 8 

#model parameters
num_filters = 128 
weight_decay = 1e-4
outputDim = 100

model = Sequential()
model.add(Embedding(input_dim=len(X_train), output_dim=outputDim))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(MaxPooling1D(2))
model.add(Conv1D(num_filters, 7, activation='relu', padding='same'))
model.add(GlobalMaxPooling1D())
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Dense(numClases, activation='sigmoid'))  #multi-label (k-hot encoding)

adam = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         15957100  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, None, 128)         89728     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, None, 128)         114816    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                4128      
__________

In [22]:
#model training
cnnmModelHist = model.fit(X_train_seq, y_train, batch_size=batch_size, epochs=num_epochs, validation_split=0.1, shuffle=True, verbose=2)

Train on 143613 samples, validate on 15958 samples
Epoch 1/8


ValueError: setting an array element with a sequence.

In [19]:
y_test_pred = model.predict(X_test_seq)

In [20]:
predicted = y_test_pred
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in tqdm(range(len(test))):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestCNN_Seq_'+ str(maxFeatures) +'.csv',encoding='utf-8', index=False)

  4%|▍         | 6067/153164 [00:09<04:00, 610.71it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [24:40<00:00, 103.45it/s]


NameError: name 'topNFeatures' is not defined

In [21]:
dfTestPredicted.to_csv('../reports/testPred/predTestCNN_Seq_'+ str(maxFeatures) +'.csv',encoding='utf-8', index=False)

In [52]:
inp = Input(shape=(len(X_train_seq[0]), ))
embed_size = 128
x = Embedding(maxFeatures, embed_size)(inp)
x = LSTM(90, return_sequences=True,name='lstm_layer')(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(60, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(7, activation="sigmoid")(x)
modelLSTM = Model(inputs=inp, outputs=x)
modelLSTM.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])


In [53]:
modelLSTM.fit(X_train_seq, y_train, batch_size=batch_size, epochs=min(num_epochs,2), validation_split=0.1);

Train on 143613 samples, validate on 15958 samples
Epoch 1/2


 98816/143613 [===================>..........] - ETA: 1:53:32 - loss: 0.6940 - acc: 0.42 - ETA: 1:32:23 - loss: 0.6911 - acc: 0.59 - ETA: 1:26:24 - loss: 0.6884 - acc: 0.69 - ETA: 1:23:13 - loss: 0.6857 - acc: 0.74 - ETA: 1:21:52 - loss: 0.6827 - acc: 0.78 - ETA: 1:21:04 - loss: 0.6796 - acc: 0.81 - ETA: 1:20:50 - loss: 0.6759 - acc: 0.83 - ETA: 1:21:10 - loss: 0.6716 - acc: 0.84 - ETA: 1:21:52 - loss: 0.6668 - acc: 0.85 - ETA: 1:22:14 - loss: 0.6604 - acc: 0.86 - ETA: 1:23:03 - loss: 0.6524 - acc: 0.86 - ETA: 1:23:46 - loss: 0.6424 - acc: 0.87 - ETA: 1:24:31 - loss: 0.6298 - acc: 0.87 - ETA: 1:25:10 - loss: 0.6165 - acc: 0.87 - ETA: 1:25:58 - loss: 0.6020 - acc: 0.88 - ETA: 1:26:36 - loss: 0.5868 - acc: 0.88 - ETA: 1:27:36 - loss: 0.5717 - acc: 0.88 - ETA: 1:28:12 - loss: 0.5565 - acc: 0.89 - ETA: 1:28:45 - loss: 0.5425 - acc: 0.89 - ETA: 1:29:14 - loss: 0.5295 - acc: 0.89 - ETA: 1:29:49 - loss: 0.5177 - acc: 0.90 - ETA: 1:31:17 - loss: 0.5049 - acc: 0.90 - ETA: 1:32:06 - loss: 0.4931

143613/143613 [==============================] - ETA: 1:26:57 - loss: 0.2131 - acc: 0.94 - ETA: 1:26:10 - loss: 0.2129 - acc: 0.94 - ETA: 1:25:22 - loss: 0.2128 - acc: 0.94 - ETA: 1:24:33 - loss: 0.2125 - acc: 0.94 - ETA: 1:23:45 - loss: 0.2123 - acc: 0.94 - ETA: 1:22:56 - loss: 0.2120 - acc: 0.94 - ETA: 1:22:06 - loss: 0.2119 - acc: 0.94 - ETA: 1:21:17 - loss: 0.2116 - acc: 0.94 - ETA: 1:20:27 - loss: 0.2114 - acc: 0.94 - ETA: 1:19:37 - loss: 0.2113 - acc: 0.94 - ETA: 1:18:46 - loss: 0.2110 - acc: 0.94 - ETA: 1:17:56 - loss: 0.2108 - acc: 0.94 - ETA: 1:17:05 - loss: 0.2106 - acc: 0.94 - ETA: 1:16:14 - loss: 0.2104 - acc: 0.94 - ETA: 1:15:22 - loss: 0.2102 - acc: 0.94 - ETA: 1:14:30 - loss: 0.2098 - acc: 0.94 - ETA: 1:13:38 - loss: 0.2096 - acc: 0.94 - ETA: 1:12:45 - loss: 0.2094 - acc: 0.94 - ETA: 1:11:52 - loss: 0.2092 - acc: 0.94 - ETA: 1:10:59 - loss: 0.2091 - acc: 0.94 - ETA: 1:10:05 - loss: 0.2089 - acc: 0.94 - ETA: 1:09:12 - loss: 0.2089 - acc: 0.94 - ETA: 1:08:17 - loss: 0.2086

 99328/143613 [===================>..........] - ETA: 8:11:53 - loss: 0.1419 - acc: 0.95 - ETA: 8:09:24 - loss: 0.1345 - acc: 0.95 - ETA: 8:07:32 - loss: 0.1419 - acc: 0.95 - ETA: 8:05:59 - loss: 0.1405 - acc: 0.95 - ETA: 8:05:04 - loss: 0.1386 - acc: 0.95 - ETA: 8:03:37 - loss: 0.1422 - acc: 0.95 - ETA: 8:01:45 - loss: 0.1433 - acc: 0.95 - ETA: 7:59:54 - loss: 0.1448 - acc: 0.95 - ETA: 7:58:15 - loss: 0.1433 - acc: 0.95 - ETA: 7:56:33 - loss: 0.1403 - acc: 0.95 - ETA: 7:54:49 - loss: 0.1386 - acc: 0.95 - ETA: 7:53:11 - loss: 0.1387 - acc: 0.95 - ETA: 7:51:31 - loss: 0.1369 - acc: 0.95 - ETA: 7:49:55 - loss: 0.1386 - acc: 0.95 - ETA: 7:48:18 - loss: 0.1375 - acc: 0.95 - ETA: 7:46:39 - loss: 0.1371 - acc: 0.95 - ETA: 7:45:08 - loss: 0.1371 - acc: 0.95 - ETA: 7:43:34 - loss: 0.1373 - acc: 0.95 - ETA: 7:42:08 - loss: 0.1358 - acc: 0.95 - ETA: 7:40:35 - loss: 0.1362 - acc: 0.95 - ETA: 7:39:00 - loss: 0.1358 - acc: 0.95 - ETA: 7:37:22 - loss: 0.1349 - acc: 0.95 - ETA: 7:35:41 - loss: 0.1351

143613/143613 [==============================] - ETA: 2:37:16 - loss: 0.1024 - acc: 0.96 - ETA: 2:35:31 - loss: 0.1023 - acc: 0.96 - ETA: 2:33:47 - loss: 0.1021 - acc: 0.96 - ETA: 2:32:02 - loss: 0.1020 - acc: 0.96 - ETA: 2:30:18 - loss: 0.1018 - acc: 0.96 - ETA: 2:28:36 - loss: 0.1016 - acc: 0.96 - ETA: 2:26:52 - loss: 0.1015 - acc: 0.96 - ETA: 2:25:00 - loss: 0.1014 - acc: 0.96 - ETA: 2:23:08 - loss: 0.1011 - acc: 0.96 - ETA: 2:21:15 - loss: 0.1009 - acc: 0.96 - ETA: 2:19:21 - loss: 0.1007 - acc: 0.96 - ETA: 2:17:28 - loss: 0.1006 - acc: 0.96 - ETA: 2:15:35 - loss: 0.1004 - acc: 0.96 - ETA: 2:13:42 - loss: 0.1003 - acc: 0.96 - ETA: 2:11:50 - loss: 0.1001 - acc: 0.96 - ETA: 2:10:01 - loss: 0.1000 - acc: 0.96 - ETA: 2:08:12 - loss: 0.0998 - acc: 0.96 - ETA: 2:06:23 - loss: 0.0996 - acc: 0.96 - ETA: 2:04:34 - loss: 0.0995 - acc: 0.96 - ETA: 2:02:46 - loss: 0.0993 - acc: 0.96 - ETA: 2:00:58 - loss: 0.0991 - acc: 0.96 - ETA: 1:59:17 - loss: 0.0989 - acc: 0.96 - ETA: 1:57:31 - loss: 0.0987

In [58]:
predicted = modelLSTM.predict(X_test_seq, batch_size=1024, verbose=1)
columns = ["id","toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
dfTestPredicted = pd.DataFrame(columns=columns)
for x in tqdm(range(len(test))):
    dfTestPredicted.loc[x] = [test['id'][x], predicted[x][1], predicted[x][2], predicted[x][3], predicted[x][4], predicted[x][5], predicted[x][6]]
dfTestPredicted.to_csv('../reports/testPred/predTestLSTM_Seq_'+ str(maxFeatures) +'.csv',encoding='utf-8', index=False)

153164/153164 [==============================] - ETA: 1:37:5 - ETA: 1:41:4 - ETA: 1:42:3 - ETA: 1:40:5 - ETA: 1:40:1 - ETA: 1:41:0 - ETA: 1:40:3 - ETA: 1:39:3 - ETA: 1:38:3 - ETA: 1:38:0 - ETA: 1:37:0 - ETA: 1:36:1 - ETA: 1:35:2 - ETA: 1:34:5 - ETA: 1:34:0 - ETA: 1:33:2 - ETA: 1:32:3 - ETA: 1:31:5 - ETA: 1:31:1 - ETA: 1:30:2 - ETA: 1:29:4 - ETA: 1:28:5 - ETA: 1:27:5 - ETA: 1:26:5 - ETA: 1:26:0 - ETA: 1:25:0 - ETA: 1:24:1 - ETA: 1:23:2 - ETA: 1:22:2 - ETA: 1:21:3 - ETA: 1:20:4 - ETA: 1:20:0 - ETA: 1:19:1 - ETA: 1:18:3 - ETA: 1:17:5 - ETA: 1:17:2 - ETA: 1:16:5 - ETA: 1:16:2 - ETA: 1:15:4 - ETA: 1:15:1 - ETA: 1:14:4 - ETA: 1:14:0 - ETA: 1:13:3 - ETA: 1:12:5 - ETA: 1:12:2 - ETA: 1:11:4 - ETA: 1:11:0 - ETA: 1:10:3 - ETA: 1:09:5 - ETA: 1:09:1 - ETA: 1:08:4 - ETA: 1:08:0 - ETA: 1:07:2 - ETA: 1:06:4 - ETA: 1:06:0 - ETA: 1:05:2 - ETA: 1:04:4 - ETA: 1:03:5 - ETA: 1:03:1 - ETA: 1:02:3 - ETA: 1:02:0 - ETA: 1:01:3 - ETA: 1:01:0 - ETA: 1:00:2 - ETA: 59:49  - ETA: 59:0 - ETA: 58:2 - ETA: 57:3 - ETA: 

  2%|▏         | 3710/153164 [00:09<06:40, 373.30it/s]C:\Anaconda2\envs\py36\lib\site-packages\tqdm\_monitor.py:89: TqdmSynchronisationWarning: Set changed size during iteration (see https://github.com/tqdm/tqdm/issues/481)
  TqdmSynchronisationWarning)
100%|██████████| 153164/153164 [48:29<00:00, 52.64it/s]


In [59]:
dfTestPredicted.to_csv('../reports/testPred/predTestLSTM_Seq_'+ str(maxFeatures) +'.csv',encoding='utf-8', index=False)

In [ ]:
# Word 2 Vec mean train list
modelWord2VecTrain = models.Word2Vec(train['listOfCleanWords'].tolist(), min_count=1)
#say_vector = modelWord2VecTrain['explanation']  # get vector for word
#say_vector

vectorizedTrainW2V = np.empty(len(train['listOfCleanWords']), dtype=list)
for i, lstOfCleanWords in enumerate(train['listOfCleanWords']): 
    size = len(lstOfCleanWords)
    if size > 0:
        vector = np.zeros(len(modelWord2VecTrain[lstOfCleanWords[0]]))
        for x in range(size):
            vector += modelWord2VecTrain[lstOfCleanWords[x]]
        vectorizedTrainW2V[i] = vector / size